# CNN training example

This notebook illustrates how to use EDIT pipeline to train a simple CNN model on the ERA5 lowres dataset.

Make sure to set the `ERA5LOWRES` environment variable to make the ERA5 low-resolution archive foundable on your system.
Modify the following cell as follows:

- for NCI

```
%env ERA5LOWRES=/g/data/wb00/NCI-Weatherbench/5.625deg
```

- for NIWA

```
%env ERA5LOWRES=/nesi/nobackup/niwa00004/riom/weatherbench/5.625deg
```

In [ ]:
%env ERA5LOWRES=/nesi/nobackup/niwa00004/riom/weatherbench/5.625deg

In [ ]:
import sys
from pathlib import Path

import numpy as np
import xarray as xr
import scores
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from lightning import Trainer, LightningModule
from lightning.pytorch.callbacks import RichProgressBar

import edit.data
import edit.tutorial
import edit.pipeline
import edit.training

In [ ]:
# train/validation/test split dates
train_start = "2015-01-01T00"
train_end = "2015-01-12T00"
val_start = "2016-01-01T00"
val_end = "2016-01-12T00"
test_start = "2017-01-01T00"
test_end = "2017-01-12T00"

# number of samples to estimate mean & standard deviation of fields
n_samples = 200
# folder to save estimated mean & standard deviation of fields
stats_folder = "cnn_training/stats"

# folders used to cache dataset processed by the pipeline
cache_folder = "cnn_training/cache"

# data loader parameters
batch_size = 1
n_workers = 2

# trainer parameters
default_root_dir = "cnn_training"
max_epochs = 10

## Data preparation

In [ ]:
data_preparation = edit.pipeline.Pipeline(
    edit.data.archive.era5lowres(["u", "v", "geopotential", "vorticity"]),
    edit.pipeline.operations.xarray.Sort(
        ["msl", "10u", "10v", "2t", "geopotential", "vorticity"]
    ),
    edit.data.transforms.coordinates.standard_longitude(type="0-360"),
    edit.pipeline.operations.xarray.reshape.CoordinateFlatten("level"),
    # retrieve previous/next samples, dt = 1H
    edit.pipeline.modifications.TemporalRetrieval(
        concat=True, samples=((-1, 1), (1, 1, 1))
    ),
    edit.pipeline.operations.xarray.conversion.ToNumpy(),
    edit.pipeline.operations.numpy.reshape.Rearrange("c t h w -> t c h w"),
    edit.pipeline.operations.numpy.reshape.Squish(axis=0),
)
data_preparation

In [ ]:
sample = data_preparation[train_start]
print(len(sample))
print(sample[0].shape)
print(sample[1].shape)

In [ ]:
train_split = edit.pipeline.iterators.DateRange(train_start, train_end, interval="1h")
train_split = train_split.randomise(seed=42)
val_split = edit.pipeline.iterators.DateRange(val_start, val_end, interval="1h")

In [ ]:
train_split[:5]

We use then use precomputed approximate mean and standard deviation using only few random samples, to rescale the input/output data to a reasonable range for model training.

In [ ]:
stats_folder = Path(stats_folder)
stats_folder.mkdir(parents=True, exist_ok=True)

mean_path = stats_folder / "mean.npy"
std_path = stats_folder / "std.npy"

# compute mean/std only if files are missing, to save time
if not mean_path.is_file() or not std_path.is_file():
    samples = np.stack([data_preparation[train_split[i]][0] for i in range(n_samples)])
    mean_approx = np.mean(samples, axis=0)
    std_approx = np.std(samples, axis=0)
    np.save(mean_path, mean_approx)
    np.save(std_path, std_approx)

In [ ]:
normaliser = edit.pipeline.operations.numpy.normalisation.Deviation(
    mean=mean_path, deviation=std_path
)
caching_step = edit.pipeline.modifications.Cache(
    cache_folder, pattern_kwargs={'extension': 'npy'}
)
data_preparation_normed = edit.pipeline.Pipeline(
    data_preparation, normaliser, caching_step
)

In [ ]:
data_preparation_normed

## Model fitting

In [ ]:
class CNN(LightningModule):
    def __init__(
        self,
        *,
        n_features: int,
        layer_sizes: list[int],
        dropout: float,
        learning_rate: float,
    ):
        super().__init__()

        layer_sizes = (n_features,) + tuple(layer_sizes)
        layers = []
        for chan_in, chan_out in zip(layer_sizes[:-1], layer_sizes[1:]):
            layers.extend(
                [
                    nn.Conv2d(chan_in, chan_out, kernel_size=3, stride=1, padding=1),
                    nn.ReLU(),
                    nn.Dropout(p=dropout),
                ]
            )
        layers.append(
            nn.Conv2d(layer_sizes[-1], n_features, kernel_size=3, stride=1, padding=1)
        )
        self.cnn = nn.Sequential(*layers)

        self.learning_rate = learning_rate
        self.loss_function = F.l1_loss

    def forward(self, x):
        return self.cnn(x)

    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        loss = self.loss_function(outputs, targets)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        loss = self.loss_function(outputs, targets)
        self.log("val_loss", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return {"optimizer": optimizer}

In [ ]:
n_features = data_preparation_normed[train_start][0].shape[-3]
model_params = dict(
    n_features=n_features, layer_sizes=[64, 64], dropout=0.6, learning_rate=1e-5
)
model = CNN(**model_params)

In [ ]:
model

In [ ]:
# ensures that we use CPU if a GPU is available, uncomment to use your GPU
%env CUDA_VISIBLE_DEVICES=

**Note:** Here we use `forkserver` to prevent deadlocks on Linux platform when using more than one worker in the data loader.

In [ ]:
data_module = edit.training.data.lightning.PipelineLightningDataModule(
    data_preparation_normed,
    train_split=train_split,
    valid_split=val_split,
    batch_size=batch_size,
    num_workers=n_workers,
    multiprocessing_context="forkserver",
    persistent_workers=True,
)

In [ ]:
data_module

In [ ]:
chkpt_path = Path(default_root_dir) / "model.ckpt"

In [ ]:
trainer = edit.training.lightning.Train(
    model,
    data_module,
    default_root_dir,
    max_epochs=max_epochs,
    callbacks=[RichProgressBar()]
)
trainer.fit(load=False)
# trainer.save(chkpt_path)

## Predictions

In [ ]:
# model = CNN.load_from_checkpoint(chkpt_path, **model_params)
model.eval()

In [ ]:
# extract a pipeline used to convert back prediction in original space
undo_pipeline = edit.pipeline.Pipeline(*data_preparation_normed.steps[-5:-1])

In [ ]:
undo_pipeline

In [ ]:
%%time
test_split = edit.pipeline.iterators.DateRange(test_start, test_end, interval="1h")

y_true = []
y_preds = []

with torch.no_grad():
    for test_date in test_split:
        X, y = data_preparation_normed[test_date]
        y_pred = model(torch.from_numpy(X)).cpu().numpy()

        y = undo_pipeline.undo(y)
        y["time"] = y.time.copy(data=[test_date.datetime64()])
        y_true.append(y)

        y_pred = undo_pipeline.undo(y_pred)
        y_pred["time"] = y_pred.time.copy(data=[test_date.datetime64()])
        y_preds.append(y_pred)

y_true = xr.concat(y_true, dim="time")
y_preds = xr.concat(y_preds, dim="time")

In [ ]:
y_preds

In [ ]:
grid = y_preds.isel(time=slice(3))["z850"].plot(col="time")
grid.fig.suptitle("Predictions", y=1.05)

In [ ]:
grid = y_true.isel(time=slice(3))["z850"].plot(col="time")
grid.fig.suptitle("Ground truth", y=1.05)

## Evaluation

In [ ]:
mae_score = scores.continuous.mae(y_preds, y_true, preserve_dims=["latitude", "longitude"])

In [ ]:
mae_score

In [ ]:
mae_score[["z500", "z850", "z1000"]].to_array(dim="field").plot(col="field")